In [5]:
import os
import gc
import glob
import random
import numpy as np 
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from scipy import stats
from pathlib import Path
from itertools import groupby
#:::::::::::::::::::::::::::::::::::
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge

folder_path = "./input"

In [6]:
de_train = pd.read_parquet(f'{folder_path}/de_train.parquet')
de_train

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,T regulatory cells,Atorvastatin,LSM-5771,CC(C)c1c(C(=O)Nc2ccccc2)c(-c2ccccc2)c(-c2ccc(F...,False,-0.014372,-0.122464,-0.456366,-0.147894,-0.545382,...,-0.549987,-2.200925,0.359806,1.073983,0.356939,-0.029603,-0.528817,0.105138,0.491015,-0.979951
610,NK cells,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,-0.455549,0.188181,0.595734,-0.100299,0.786192,...,-1.236905,0.003854,-0.197569,-0.175307,0.101391,1.028394,0.034144,-0.231642,1.023994,-0.064760
611,T cells CD4+,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,0.338168,-0.109079,0.270182,-0.436586,-0.069476,...,0.077579,-1.101637,0.457201,0.535184,-0.198404,-0.005004,0.552810,-0.209077,0.389751,-0.337082
612,T cells CD8+,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,0.101138,-0.409724,-0.606292,-0.071300,-0.001789,...,0.005951,-0.893093,-1.003029,-0.080367,-0.076604,0.024849,0.012862,-0.029684,0.005506,-1.733112


In [8]:
id_map = pd.read_csv(f'{folder_path}/id_map.csv', index_col='id')
id_map

,cell_type,sm_name
id,,
0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...
1,B cells,ABT-199 (GDC-0199)
2,B cells,ABT737
3,B cells,AMD-070 (hydrochloride)
4,B cells,AT 7867
...,...,...
250,Myeloid cells,Vandetanib
251,Myeloid cells,Vanoxerine
252,Myeloid cells,Vardenafil


In [9]:
genes = de_train.columns[5:]
genes

Index(['A1BG', 'A1BG-AS1', 'A2M', 'A2M-AS1', 'A2MP1', 'A4GALT', 'AAAS', 'AACS',
       'AAGAB', 'AAK1',
       ...
       'ZUP1', 'ZW10', 'ZWILCH', 'ZWINT', 'ZXDA', 'ZXDB', 'ZXDC', 'ZYG11B',
       'ZYX', 'ZZEF1'],
      dtype='object', length=18211)

In [10]:
def add_columns(de_train, id_map):
    sm_lincs_id = de_train.set_index('sm_name')["sm_lincs_id"].to_dict()
    sm_name_to_smiles = de_train.set_index('sm_name')['SMILES'].to_dict()

    id_map['sm_lincs_id'] = id_map['sm_name'].map(sm_lincs_id)
    id_map['SMILES'] = id_map['sm_name'].map(sm_name_to_smiles)
    
    return id_map

add_columns(de_train, id_map)

,cell_type,sm_name,sm_lincs_id,SMILES
id,,,,
0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,LSM-47134,Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C
1,B cells,ABT-199 (GDC-0199),LSM-45468,CC1(C)CCC(CN2CCN(c3ccc(C(=O)NS(=O)(=O)c4ccc(NC...
2,B cells,ABT737,LSM-1180,CN(C)CC[C@H](CSc1ccccc1)Nc1ccc(S(=O)(=O)NC(=O)...
3,B cells,AMD-070 (hydrochloride),LSM-45591,NCCCCN(Cc1nc2ccccc2[nH]1)[C@H]1CCCc2cccnc21
4,B cells,AT 7867,LSM-1155,Clc1ccc(C2(c3ccc(-c4cn[nH]c4)cc3)CCNCC2)cc1
...,...,...,...,...
250,Myeloid cells,Vandetanib,LSM-1199,COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1
251,Myeloid cells,Vanoxerine,LSM-2703,Fc1ccc(C(OCCN2CCN(CCCc3ccccc3)CC2)c2ccc(F)cc2)cc1
252,Myeloid cells,Vardenafil,LSM-2292,CCCc1nc(C)c2c(=O)[nH]c(-c3cc(S(=O)(=O)N4CCN(CC...


In [24]:
def calculate_mae_and_mrrmse(y_true, y_pred, scaler=None):
    if scaler:
        y_true = scaler.inverse_transform(y_true)
        y_pred = scaler.inverse_transform(y_pred)

        # Calculate Mean Rowwise Root Mean Squared Error (MRRMSE)
        rowwise_rmse = np.sqrt(np.mean(np.square(y_true - y_pred), axis=1))
        mrrmse_score = np.mean(rowwise_rmse)
    else:
        # Calculate Mean Rowwise Root Mean Squared Error (MRRMSE)
        rowwise_rmse = np.sqrt(np.mean(np.square(y_true - y_pred), axis=1))
        mrrmse_score = np.mean(rowwise_rmse)
    
    # Print the results
    print(f"Mean Rowwise Root Mean Squared Error (MRRMSE): {mrrmse_score}")

In [12]:
kf = KFold(n_splits=6, shuffle=True, random_state=6174)

# Create k-fold columns
for fold, (train_index, test_index) in enumerate(kf.split(de_train)):
    de_train.loc[test_index, 'kfold'] = fold

de_train['kfold'] = de_train['kfold'].astype(int)

# Reindex DataFrame to put 'kfold' as the first column
cols = ['kfold'] + [col for col in de_train if col != 'kfold']
de_train = de_train[cols]

de_train

,kfold,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,4,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,5,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,4,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,5,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,1,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,4,T regulatory cells,Atorvastatin,LSM-5771,CC(C)c1c(C(=O)Nc2ccccc2)c(-c2ccccc2)c(-c2ccc(F...,False,-0.014372,-0.122464,-0.456366,-0.147894,...,-0.549987,-2.200925,0.359806,1.073983,0.356939,-0.029603,-0.528817,0.105138,0.491015,-0.979951
610,4,NK cells,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,-0.455549,0.188181,0.595734,-0.100299,...,-1.236905,0.003854,-0.197569,-0.175307,0.101391,1.028394,0.034144,-0.231642,1.023994,-0.064760
611,4,T cells CD4+,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,0.338168,-0.109079,0.270182,-0.436586,...,0.077579,-1.101637,0.457201,0.535184,-0.198404,-0.005004,0.552810,-0.209077,0.389751,-0.337082
612,2,T cells CD8+,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,0.101138,-0.409724,-0.606292,-0.071300,...,0.005951,-0.893093,-1.003029,-0.080367,-0.076604,0.024849,0.012862,-0.029684,0.005506,-1.733112


In [16]:
test_df = de_train[de_train["kfold"]==0].sample(frac=1, random_state=6174)
train_df = de_train[de_train["kfold"]!=0].sample(frac=1, random_state=6174)
train_df

/var/folders/97/ycz7vyts3nb9l4w_17fp37800000gn/T/ipykernel_9391/2538463333.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  de_train.drop(test_df.index, inplace=True)


,kfold,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,4,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,5,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,4,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,5,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,1,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,4,T regulatory cells,Atorvastatin,LSM-5771,CC(C)c1c(C(=O)Nc2ccccc2)c(-c2ccccc2)c(-c2ccc(F...,False,-0.014372,-0.122464,-0.456366,-0.147894,...,-0.549987,-2.200925,0.359806,1.073983,0.356939,-0.029603,-0.528817,0.105138,0.491015,-0.979951
610,4,NK cells,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,-0.455549,0.188181,0.595734,-0.100299,...,-1.236905,0.003854,-0.197569,-0.175307,0.101391,1.028394,0.034144,-0.231642,1.023994,-0.064760
611,4,T cells CD4+,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,0.338168,-0.109079,0.270182,-0.436586,...,0.077579,-1.101637,0.457201,0.535184,-0.198404,-0.005004,0.552810,-0.209077,0.389751,-0.337082
612,2,T cells CD8+,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,0.101138,-0.409724,-0.606292,-0.071300,...,0.005951,-0.893093,-1.003029,-0.080367,-0.076604,0.024849,0.012862,-0.029684,0.005506,-1.733112


In [84]:
for i in range(1, 6):
		train_index = de_train[de_train["kfold"]!=i].index
		val_index = de_train[de_train["kfold"]==i].index

		train_df = de_train.loc[train_index].copy()
		val_df = de_train.loc[val_index].copy()

		# genes
		pred_df = val_df.copy()
		pred_df[genes] = 0
		for sm_name in val_df["sm_name"].unique():
				pred_df.loc[val_df["sm_name"]==sm_name, genes] = train_df[train_df["sm_name"]==sm_name][genes].mean().to_list()
		
		# Fillna by mean
		pred_df.fillna(pred_df[genes].mean(), inplace=True)

		# Get validation score
		calculate_mae_and_mrrmse(val_df[genes].values, pred_df[genes].values)

Mean Rowwise Root Mean Squared Error (MRRMSE): 1.4257857607588547
Mean Rowwise Root Mean Squared Error (MRRMSE): 1.5634813004834938
Mean Rowwise Root Mean Squared Error (MRRMSE): 1.2125476143984675
Mean Rowwise Root Mean Squared Error (MRRMSE): 1.4282084637326142
Mean Rowwise Root Mean Squared Error (MRRMSE): 1.3916449898439396


In [90]:
import joblib

features = ['cell_type', 'sm_name']

n_components = 100
alpha = 5


for i in range(1, 6):
    train_index = train_data[train_data["kfold"]!=i].index
    val_index = train_data[train_data["kfold"]==i].index

    train_df = train_data.loc[train_index].sample(frac=1, random_state=6174)
    val_df = train_data.loc[val_index].sample(frac=1, random_state=6174)

    # genes
    pred_df = val_df.copy()
    pred_df[genes] = 0

    # Model
    model = make_pipeline(
        ColumnTransformer([('ohe', OneHotEncoder(handle_unknown='ignore'), ['sm_name'])]),
        Ridge(alpha=alpha, fit_intercept=False)
    )
    svd = TruncatedSVD(n_components=n_components, random_state=6174)
    z_tr = svd.fit_transform(train_df[genes])
    model.fit(train_df[features], z_tr)
    joblib.dump(model, f'sklearn_pipeline_{i}.pkl')
    y_pred = svd.inverse_transform(model.predict(val_df[features]))
    pred_df[genes] = y_pred

    # Get validation score
    calculate_mae_and_mrrmse(val_df[genes].values, pred_df[genes].values)

AttributeError: 'Pipeline' object has no attribute 'best_estimator_'

In [ ]:
n_components = 100
alpha = 5

val_preds = np.array([])
test_preds = np.array([])
for i in range(5):
	val_df = de_train.loc[val_index[i]].copy()
	train_df = de_train.loc[~de_train.index.isin(val_index[i])].copy()

	train_pca = pca_train.loc[~pca_train.index.isin(val_index[i])].copy()
	val_pca = pca_train.loc[val_index[i]].copy()

	pred_df = val_df.copy()
	pred_df[genes] = 0

	# Model
	model = make_pipeline(
		TruncatedSVD(n_components=n_components, random_state=6174),
		LinearRegression()
	)
	svd = TruncatedSVD(n_components=n_components, random_state=6174)
	z_tr = svd.fit_transform(train_df[genes])
	model.fit(train_pca, z_tr)

	val_pred = svd.inverse_transform(model.predict(val_pca))
	val_preds = np.append(val_preds, val_pred)
	print(f'Fold {i} MRRMSE: {mrrmse_pd(val_pred, val_df[genes]):.6f}')

	test_pred = svd.inverse_transform(model.predict(pca_test))


val_pred = val_preds.reshape(5, -1)
val_pred = np.mean(val_pred, axis=0)

print(f'CV MRRMSE: {mrrmse_pd(val_preds, de_train[genes]):.6f}')

test_preds = np.mean(test_preds, axis=0)
id_map.loc[:, genes] = test_preds
id_map = id_map.loc[:, genes]
id_map.to_csv('submission.csv')
id_map

In [ ]:
consistent_val_preds = []
for pred in val_preds:
    arr = np.array(pred)
    # Truncate if longer
    if arr.shape > target_shape:
        arr = arr[:target_shape]
    # Pad with zeros (or any other appropriate value) if shorter
    elif arr.shape < target_shape:
        arr = np.pad(arr, (0, target_shape - arr.shape), mode='constant')
    consistent_val_preds.append(arr)

# Now you can compute the mean
val_pred = np.mean(consistent_val_preds, axis=0)